# Фина настройка на модели на Open AI

Този бележник е базиран на текущите указания, предоставени в документацията за [Фина настройка](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) от Open AI.

Фината настройка подобрява представянето на основните модели за вашето приложение чрез повторно обучение с допълнителни данни и контекст, релевантни за конкретния случай на употреба или сценарий. Обърнете внимание, че техники за инженеринг на подканите като _few shot learning_ и _retrieval augmented generation_ ви позволяват да подобрите стандартната подканваща фраза с релевантни данни за повишаване на качеството. Въпреки това, тези подходи са ограничени от максималния размер на прозореца за токени на целевия основен модел.

С фината настройка ефективно преобучаваме самия модел с необходимите данни (което ни позволява да използваме много повече примери, отколкото могат да се поберат в максималния прозорец за токени) - и внедряваме _персонализирана_ версия на модела, която вече не се нуждае от предоставяне на примери по време на извеждане. Това не само подобрява ефективността на дизайна на подканите ни (имаме повече гъвкавост при използването на прозореца за токени за други неща), но потенциално подобрява и разходите ни (чрез намаляване на броя токени, които трябва да изпращаме към модела по време на извеждане).

Фината настройка има 4 стъпки:
1. Подгответе тренировъчните данни и ги качете.
1. Стартирайте тренировъчната задача, за да получите фино настроен модел.
1. Оценете фино настроения модел и итеративно подобрявайте качеството.
1. Внедрете фино настроения модел за извеждане, когато сте доволни.

Обърнете внимание, че не всички основни модели поддържат фина настройка - [проверете документацията на OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) за най-актуална информация. Можете също да фино настроите вече фино настроен модел. В този урок ще използваме `gpt-35-turbo` като наш целеви основен модел за фина настройка.

---


### Стъпка 1.1: Подгответе своя набор от данни

Нека създадем чатбот, който да ви помогне да разберете периодичната таблица на елементите, като отговаря на въпроси за даден елемент с лимерик. В _този_ прост урок ще създадем само набор от данни, за да обучим модела с няколко примерни отговора, които показват очаквания формат на данните. В реален случай на употреба ще трябва да създадете набор от данни с много повече примери. Може също да използвате отворен набор от данни (за вашата област на приложение), ако такъв съществува, и да го преформатирате за използване при фино настройване.

Тъй като се фокусираме върху `gpt-35-turbo` и търсим еднооборотен отговор (чат завършек), можем да създадем примери, използвайки [този предложен формат](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst), който отразява изискванията за чат завършек на OpenAI. Ако очаквате многооборотно разговорно съдържание, ще използвате [формата за многооборотни примери](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst), който включва параметър `weight`, за да сигнализира кои съобщения трябва да се използват (или не) в процеса на фино настройване.

Ще използваме по-простия еднооборотен формат за нашия урок тук. Данните са във [jsonl формат](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) с 1 запис на ред, всеки представен като JSON-обект. Фрагментът по-долу показва 2 записа като пример - вижте [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) за пълния примерен набор (10 примера), който ще използваме за нашия урок по фино настройване. **Забележка:** Всеки запис _трябва_ да бъде дефиниран в един ред (не разделен на няколко реда, както е типично за форматиран JSON файл)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

В реален случай на употреба ще ви трябва много по-голям набор от примери за добри резултати - компромисът ще бъде между качеството на отговорите и времето/разходите за фино настройване. Ние използваме малък набор, за да можем бързо да завършим финото настройване и да илюстрираме процеса. Вижте [този пример от OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) за по-сложен урок по фино настройване.


---

### Стъпка 1.2 Качване на вашия набор от данни

Качете данните, използвайки Files API [както е описано тук](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Обърнете внимание, че за да изпълните този код, трябва първо да сте направили следните стъпки:
 - Инсталирали сте Python пакета `openai` (уверете се, че използвате версия >=0.28.0 за най-новите функции)
 - Зададете променливата на средата `OPENAI_API_KEY` със своя OpenAI API ключ
За да научите повече, вижте [Ръководството за настройка](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst), предоставено за курса.

Сега изпълнете кода, за да създадете файл за качване от вашия локален JSONL файл.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### Стъпка 2.1: Създаване на задача за фино настройване с SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### Стъпка 2.2: Проверете състоянието на задачата

Ето няколко неща, които можете да направите с API-то `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - Изброява последните n задачи за фино настройване
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Вземете подробности за конкретна задача за фино настройване
- `client.fine_tuning.jobs.cancel(<job_id>)` - Отменете задача за фино настройване
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Изброява до n събития от задачата
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Първата стъпка от процеса е _валидиране на тренировъчния файл_, за да се уверите, че данните са в правилния формат.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### Стъпка 2.3: Следете събитията, за да наблюдавате напредъка


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Стъпка 2.4: Преглед на статуса в OpenAI таблото за управление


Можете също да видите статуса, като посетите уебсайта на OpenAI и разгледате секцията _Фина настройка_ на платформата. Там ще видите статуса на текущата задача, както и ще можете да проследите историята на предишните изпълнения на задачите. На този екран можете да видите, че предишното изпълнение е неуспешно, а второто изпълнение е успешно. За контекст, това се случи, когато първото изпълнение използва JSON файл с неправилно форматирани записи - след като беше поправено, второто изпълнение завърши успешно и направи модела достъпен за използване.

![Fine-tuning job status](../../../../../translated_images/bg/fine-tuned-model-status.563271727bf7bfba.png)


Можете също да видите съобщенията за състоянието и метриките, като превъртите по-надолу в визуалния табло, както е показано:

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/bg/fine-tuned-messages-panel.4ed0c2da5ea1313b.png) |  ![Metrics](../../../../../translated_images/bg/fine-tuned-metrics-panel.700d7e4995a65229.png)|


---

### Стъпка 3.1: Вземете ID и тествайте фино настроения модел в кода


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### Стъпка 3.2: Зареждане и тестване на фино настроен модел в Playground

Сега можете да тествате фино настроения модел по два начина. Първо, можете да посетите Playground и да използвате падащото меню Models, за да изберете новия си фино настроен модел от изброените опции. Другата опция е да използвате опцията "Playground", показана в панела за фино настройване (вижте екранната снимка по-горе), която стартира следния _сравнителен_ изглед, който показва основния и фино настроения модел един до друг за бърза оценка.

![Fine-tuning job status](../../../../../translated_images/bg/fine-tuned-playground-compare.56e06f0ad8922016.png)

Просто попълнете системния контекст, използван в данните за обучение, и задайте своя тестов въпрос. Ще забележите, че и двете страни се актуализират с идентичен контекст и въпрос. Стартирайте сравнението и ще видите разликата в изходите между тях. _Обърнете внимание как фино настроеният модел генерира отговора във формата, който сте предоставили в примерите си, докато основният модел просто следва системната подсказка_.

![Fine-tuning job status](../../../../../translated_images/bg/fine-tuned-playground-launch.5a26495c983c6350.png)

Ще забележите, че сравнението също така предоставя броя на токените за всеки модел и времето, необходимо за извеждане на резултата. **Този конкретен пример е опростен и има за цел да покаже процеса, но не отразява реален набор от данни или сценарий**. Може да забележите, че и двата примера показват същия брой токени (системният контекст и потребителската подсказка са идентични), като фино настроеният модел отнема повече време за извеждане на резултата (персонализиран модел).

В реални сценарии няма да използвате такъв опростен пример, а ще правите фино настройване с реални данни (например продуктов каталог за обслужване на клиенти), където качеството на отговора ще бъде много по-очевидно. В _този_ контекст, постигането на еквивалентно качество на отговора с основния модел ще изисква повече персонализиране на подсказките, което ще увеличи използването на токени и потенциално свързаното време за обработка при извеждане на резултата. _За да изпробвате това, разгледайте примерите за фино настройване в OpenAI Cookbook, за да започнете._

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Отказ от отговорност**:  
Този документ е преведен с помощта на AI преводаческа услуга [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи могат да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Ние не носим отговорност за каквито и да е недоразумения или неправилни тълкувания, произтичащи от използването на този превод.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
